In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import json
import os
import scipy.signal

from spykshrk.realtime.decoder_process import PointProcessDecoder

from spykshrk.realtime.simulator import nspike_data

from spykshrk.franklab.pp_decoder.util import gaussian, normal2D, apply_no_anim_boundary, simplify_pos_pandas
from spykshrk.franklab.pp_decoder.pp_clusterless import OfflinePPDecoder
from spykshrk.franklab.pp_decoder.data_containers import EncodeSettings, DecodeSettings, SpikeObservation, \
                                                         LinearPosition, StimLockout, Posteriors
from spykshrk.franklab.pp_decoder.visualization import DecodeVisualizer

from spykshrk.franklab.pp_decoder.decode_error import bin_pos_data, calc_error_table, \
                                                      plot_arms_error

#pd.set_option('float_format', '{:,.2f}'.format)
pd.set_option('display.precision', 4)
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 15)
#pd.set_option('display.width', 180)


idx = pd.IndexSlice

In [2]:
%%time
# Load merged rec HDF store based on config

config_file = '/opt/data36/daliu/realtime/spykshrk/dec_60uv_300samp/bond.config.json'
#config_file = '/home/daliu/Src/spykshrk_realtime/config/bond_single.json'

config = json.load(open(config_file, 'r'))


# Main hdf5 data source file name
hdf_file = os.path.join(config['files']['output_dir'],
                        '{}.rec_merged.h5'.format(config['files']['prefix']))

# Extract just encode and decode settings from config
encode_settings = EncodeSettings(config)
decode_settings = DecodeSettings(config)

# Open data file
store = pd.HDFStore(hdf_file, mode='r')

# Encapsulate Spike Observation panda table in container
observ_obj = SpikeObservation.create_default(store['rec_3'])

# Grab stimulation lockout times
stim_lockout = StimLockout.create_default(store['rec_11'])


# Grab animal linearized real position
nspike_anim = nspike_data.AnimalInfo(**config['simulator']['nspike_animal_info'])
pos = nspike_data.PosMatDataStream(nspike_anim)
pos_data = pos.data

# Encapsulate linear position
lin_obj = LinearPosition.from_nspike_posmat(pos_data, encode_settings)

In [3]:
%%time
# Run PP decoding algorithm

time_bin_size = 300

decoder = OfflinePPDecoder(lin_obj=lin_obj, observ_obj=observ_obj,
                           encode_settings=encode_settings, decode_settings=decode_settings, 
                           which_trans_mat='learned')

posteriors = decoder.run_decoder()

In [4]:
%%time
# Run spykshrk.realtime version of point process decoding

# Create and setup online point process decoder
pp_decoder = PointProcessDecoder(pos_range=[config['encoder']['position']['lower'],
                                            config['encoder']['position']['upper']],
                                 pos_bins=config['encoder']['position']['bins'],
                                 time_bin_size=config['pp_decoder']['bin_size'],
                                 arm_coor=config['encoder']['position']['arm_pos'],
                                 uniform_gain=config['pp_decoder']['trans_mat_uniform_gain'])

pp_decoder.select_ntrodes(config['simulator']['nspike_animal_info']['tetrodes'])

observ_obj.update_observations_bins(time_bin_size)

num_time_bins = observ_obj.index.get_level_values('dec_bin').max()

# Group by bin
groups = observ_obj.groupby('dec_bin')

last_bin_id = 0

spykshrk_posteriors = np.zeros([num_time_bins+1, config['encoder']['position']['bins']])

for bin_id, spikes_in_bin in groups:
    if last_bin_id <= bin_id - 1:
        # increment bins with no spikes
        for bin_no_spk_id in range(last_bin_id + 1, bin_id):
            post = pp_decoder.increment_no_spike_bin()
            spykshrk_posteriors[bin_no_spk_id, :] = post
        
    # Add 
    for ntrode_id, dec in zip(spikes_in_bin.loc[:, 'ntrode_id'].values, 
                   spikes_in_bin.loc[:, 'x000': 'x{:03d}'.
                                     format(config['encoder']['position']['bins']-1)].values):
        pp_decoder.add_observation(ntrode_id, dec)
        
    post = pp_decoder.increment_bin()
    spykshrk_posteriors[bin_id, :] = post
    last_bin_id = bin_id
    
spykshrk_posteriors = Posteriors.from_numpy(spykshrk_posteriors, index=posteriors.index, columns=
                                           ['x{:03d}'.format(pos_ind) for pos_ind in 
                                            range(config['encoder']['position']['bins'])])

In [8]:
spykshrk_posteriors

In [6]:
## Plot posteriors

plt_ranges = [[2461 + 250, 2461 + 400]]
             
for plt_range in plt_ranges:
    
    plt.figure(figsize=[200,10])
    plt.subplot(2,1,1)
    ax1 = DecodeVisualizer.plot_decode_image(spykshrk_posteriors, plt_range, encode_settings)
    DecodeVisualizer.plot_linear_pos(lin_obj, plt_range)
    DecodeVisualizer.plot_stim_lockout(stim_lockout, plt_range, encode_settings.arm_coordinates[2][1] + 10)

    plt.subplot(2,1,2)
    ax1 = DecodeVisualizer.plot_decode_image(posteriors, plt_range, encode_settings)
    DecodeVisualizer.plot_linear_pos(lin_obj, plt_range)
    DecodeVisualizer.plot_stim_lockout(stim_lockout, plt_range, encode_settings.arm_coordinates[2][1] + 10)
    
    plt.xlim(plt_range)
plt.show()

In [7]:
spyk_dec_est_map = x_bins[np.argmax(spykshrk_posteriors, axis=1)]
spyk_dec_est_pos = pd.DataFrame({'est_pos': spyk_dec_est_map}, 
                                index=pd.Index(data=dec_bin_times, name='timestamp'))

pos_data_bins = bin_pos_data(pos_data_linpos, dec_bin_size)

spyk_center_dec_error, spyk_left_dec_error, spyk_right_dec_error = calc_error_table(pos_data_bins,
                                                                                    spyk_dec_est_pos,
                                                                                    [[0, 70], 
                                                                                     [149, 256], 
                                                                                     [290, 407]],
                                                                                    2)

In [ ]:
print('median error center: {:0.5}, left: {:0.5}, right: {:.5}'.format(np.median(spyk_center_dec_error['abs_error']),
                                                                       np.median(spyk_left_dec_error['abs_error']),
                                                                       np.median(spyk_right_dec_error['abs_error'])))

print('mean error center: {:0.5}, left: {:0.5}, right: {:.5}'.format(np.mean(spyk_center_dec_error['abs_error']),
                                                                     np.mean(spyk_left_dec_error['abs_error']),
                                                                     np.mean(spyk_right_dec_error['abs_error'])))

In [ ]:
plot_arms_error(spyk_center_dec_error, spyk_left_dec_error, spyk_right_dec_error, [2350, 3400])
plt.show()